# fetch

In [ ]:
import ccxt
import pandas as pd
import os
import time

exchange = ccxt.hyperliquid()

markets = exchange.fetch_markets()

spot_pairs = [market['symbol'] for market in markets if market['type'] == 'spot']
spot_count = len(spot_pairs)

print(f"Hyperliquid spot pair num: {spot_count}")

folder_name = 'hyperliquid_spot'
os.makedirs(folder_name, exist_ok=True)


timeframe = '1d'
since = exchange.parse8601('2023-01-01T00:00:00Z')


for symbol in spot_pairs:
    all_ohlcv = []

   
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since)
   
    if ohlcv:
        all_ohlcv.extend(ohlcv)

        ohlcv_df = pd.DataFrame(all_ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        ohlcv_df['timestamp'] = pd.to_datetime(ohlcv_df['timestamp'], unit='ms')

        file_name = os.path.join(folder_name, f"{symbol.replace('/', '_')}_ohlcv.csv")
        ohlcv_df.to_csv(file_name, index=False)

        print(f"Saved OHLCV data for {symbol} to {file_name}")
    else:
        print(f"No data available for {symbol}")

    print("Waiting for 5 seconds...")
    time.sleep(5)

In [16]:
import os

# 打印当前工作目录
print("Current directory:", os.getcwd())
import os

# 列出当前工作目录中的所有文件和文件夹
files = os.listdir()
print("Files and folders in current directory:", files)

Current directory: /workspaces/Hyperliquid
Files and folders in current directory: ['spot', 'README.md', '.git']


In [15]:
spot_folder = 'spot'

if os.path.exists(spot_folder):
    contents = os.listdir(spot_folder)
    print(f"Contents of '{spot_folder}':", contents)
else:
    print(f"'{spot_folder}' folder not found.")

Contents of 'spot': ['dec_17', 'spot.ipynb']


In [4]:
import pandas as pd
import os


# 读取 OHLCV 数据
folder_name = 'spot/hyperliquid_spot'
files = os.listdir(folder_name)

# 创建一个 DataFrame 存储结果
results = []

for file in files:
    if file.endswith('_ohlcv.csv'):
        # 读取 CSV 文件
        file_path = os.path.join(folder_name, file)
        ohlcv_df = pd.read_csv(file_path)

        # 计算 Token symbol，只保留下划线前的部分
        token_symbol = file.split('_')[0]  # 提取第一个部分，例如 'ADHD'

        # 找到上市日期（第一次出现价格的时间）
        ohlcv_df['timestamp'] = pd.to_datetime(ohlcv_df['timestamp'])
        listed_date = ohlcv_df['timestamp'].iloc[0]

        # 计算价格变化
        price_change = {
            '1d': ohlcv_df.loc[ohlcv_df['timestamp'] > listed_date + pd.Timedelta(days=1), 'close'].iloc[0] / ohlcv_df['close'].iloc[0] - 1,
            '30d': ohlcv_df.loc[ohlcv_df['timestamp'] > listed_date + pd.Timedelta(days=30), 'close'].iloc[0] / ohlcv_df['close'].iloc[0] - 1,
            '60d': ohlcv_df.loc[ohlcv_df['timestamp'] > listed_date + pd.Timedelta(days=60), 'close'].iloc[0] / ohlcv_df['close'].iloc[0] - 1,
        }

        # 计算 ATH 和日期
        ath_price = ohlcv_df['close'].max()
        ath_date = ohlcv_df.loc[ohlcv_df['close'].idxmax(), 'timestamp']
        days_to_ath = (ath_date - listed_date).days

        # 计算交易量变化
        vol_change = {
            '1d': ohlcv_df.loc[ohlcv_df['timestamp'] > listed_date + pd.Timedelta(days=1), 'volume'].iloc[0] / ohlcv_df['volume'].iloc[0] - 1,
            '30d': ohlcv_df.loc[ohlcv_df['timestamp'] > listed_date + pd.Timedelta(days=30), 'volume'].iloc[0] / ohlcv_df['volume'].iloc[0] - 1,
            '60d': ohlcv_df.loc[ohlcv_df['timestamp'] > listed_date + pd.Timedelta(days=60), 'volume'].iloc[0] / ohlcv_df['volume'].iloc[0] - 1,
        }

        # 收集结果
        results.append({
            'Token symbol': token_symbol,
            'Listed date': listed_date,
            'ATH date': ath_date,  # 添加 ATH 日期
            'Price % change 1d': price_change['1d'] * 100,
            'Price % change 30d': price_change['30d'] * 100,
            'Price % change 60d': price_change['60d'] * 100,
            'ATH %': (ath_price / ohlcv_df['close'].iloc[0] - 1) * 100,
            'Day to ATH': days_to_ath,
            'Vol % change 1d': vol_change['1d'] * 100,
            'Vol % change 30d': vol_change['30d'] * 100,
            'Vol % change 60d': vol_change['60d'] * 100,
        })

# 创建最终的 DataFrame
results_df = pd.DataFrame(results)

# 保存结果到 CSV
results_df.to_csv('hyperliquid_spot_summary.csv', index=False)

print("汇总数据已保存到 hyperliquid_spot_summary.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'spot/hyperliquid_spot'